# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 16 2022 2:08PM,246635,21,409206606,"NBTY Global, Inc.",Released
1,Sep 16 2022 2:07PM,246638,15,PNC381367A,"Person & Covey, Inc.",Released
2,Sep 16 2022 2:07PM,246638,15,PNC381379A,"Person & Covey, Inc.",Released
3,Sep 16 2022 2:07PM,246638,15,PNC370892D,"Person & Covey, Inc.",Released
4,Sep 16 2022 2:07PM,246638,15,PNC384506A,"Person & Covey, Inc.",Released
5,Sep 16 2022 2:07PM,246638,15,PNC384560A,"Person & Covey, Inc.",Released
6,Sep 16 2022 2:07PM,246638,15,PNC384564A,"Person & Covey, Inc.",Released
7,Sep 16 2022 2:07PM,246638,15,PNC384575A,"Person & Covey, Inc.",Released
8,Sep 16 2022 2:07PM,246638,15,PNC384821A,"Person & Covey, Inc.",Released
9,Sep 16 2022 2:07PM,246638,15,PNC384833A,"Person & Covey, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
16,246638,Released,17
17,246639,Released,5
18,246640,Released,1
19,246641,Released,3
20,246642,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
246638,NaN,17.0
246639,NaN,5.0
246640,NaN,1.0
246641,NaN,3.0
246642,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
246578,1.0,1.0
246595,0.0,1.0
246597,0.0,1.0
246598,7.0,3.0
246612,2.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
246578,1,1
246595,0,1
246597,0,1
246598,7,3
246612,2,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,246578,1,1
1,246595,0,1
2,246597,0,1
3,246598,7,3
4,246612,2,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,246578,1,1
1,246595,,1
2,246597,,1
3,246598,7,3
4,246612,2,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 16 2022 2:08PM,246635,21,"NBTY Global, Inc."
1,Sep 16 2022 2:07PM,246638,15,"Person & Covey, Inc."
18,Sep 16 2022 1:59PM,246642,16,"SHL Pharma, LLC"
19,Sep 16 2022 1:54PM,246641,10,Emerginnova
22,Sep 16 2022 1:54PM,246640,16,"SHL Pharma, LLC"
23,Sep 16 2022 1:52PM,246639,10,ISDIN Corporation
28,Sep 16 2022 1:46PM,246637,10,ISDIN Corporation
39,Sep 16 2022 1:00PM,246632,20,Alumier Labs Inc.
41,Sep 16 2022 12:57PM,246631,20,Alumier Labs Inc.
43,Sep 16 2022 12:55PM,246630,20,Alumier Labs Inc.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Sep 16 2022 2:08PM,246635,21,"NBTY Global, Inc.",,1
1,Sep 16 2022 2:07PM,246638,15,"Person & Covey, Inc.",,17
2,Sep 16 2022 1:59PM,246642,16,"SHL Pharma, LLC",,1
3,Sep 16 2022 1:54PM,246641,10,Emerginnova,,3
4,Sep 16 2022 1:54PM,246640,16,"SHL Pharma, LLC",,1
5,Sep 16 2022 1:52PM,246639,10,ISDIN Corporation,,5
6,Sep 16 2022 1:46PM,246637,10,ISDIN Corporation,,11
7,Sep 16 2022 1:00PM,246632,20,Alumier Labs Inc.,,2
8,Sep 16 2022 12:57PM,246631,20,Alumier Labs Inc.,1,1
9,Sep 16 2022 12:55PM,246630,20,Alumier Labs Inc.,2,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Sep 16 2022 2:08PM,246635,21,"NBTY Global, Inc.",1,
1,Sep 16 2022 2:07PM,246638,15,"Person & Covey, Inc.",17,
2,Sep 16 2022 1:59PM,246642,16,"SHL Pharma, LLC",1,
3,Sep 16 2022 1:54PM,246641,10,Emerginnova,3,
4,Sep 16 2022 1:54PM,246640,16,"SHL Pharma, LLC",1,
5,Sep 16 2022 1:52PM,246639,10,ISDIN Corporation,5,
6,Sep 16 2022 1:46PM,246637,10,ISDIN Corporation,11,
7,Sep 16 2022 1:00PM,246632,20,Alumier Labs Inc.,2,
8,Sep 16 2022 12:57PM,246631,20,Alumier Labs Inc.,1,1
9,Sep 16 2022 12:55PM,246630,20,Alumier Labs Inc.,1,2


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Sep 16 2022 2:08PM,246635,21,"NBTY Global, Inc.",1,
1,Sep 16 2022 2:07PM,246638,15,"Person & Covey, Inc.",17,
2,Sep 16 2022 1:59PM,246642,16,"SHL Pharma, LLC",1,
3,Sep 16 2022 1:54PM,246641,10,Emerginnova,3,
4,Sep 16 2022 1:54PM,246640,16,"SHL Pharma, LLC",1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Sep 16 2022 2:08PM,246635,21,"NBTY Global, Inc.",1.0,NaN
1,Sep 16 2022 2:07PM,246638,15,"Person & Covey, Inc.",17.0,NaN
2,Sep 16 2022 1:59PM,246642,16,"SHL Pharma, LLC",1.0,NaN
3,Sep 16 2022 1:54PM,246641,10,Emerginnova,3.0,NaN
4,Sep 16 2022 1:54PM,246640,16,"SHL Pharma, LLC",1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Sep 16 2022 2:08PM,246635,21,"NBTY Global, Inc.",1.0,0.0
1,Sep 16 2022 2:07PM,246638,15,"Person & Covey, Inc.",17.0,0.0
2,Sep 16 2022 1:59PM,246642,16,"SHL Pharma, LLC",1.0,0.0
3,Sep 16 2022 1:54PM,246641,10,Emerginnova,3.0,0.0
4,Sep 16 2022 1:54PM,246640,16,"SHL Pharma, LLC",1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,1233133,38.0,7.0
15,246638,17.0,0.0
16,1233052,5.0,1.0
20,986505,4.0,5.0
21,246635,1.0,0.0
22,246629,0.0,1.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,1233133,38.0,7.0
1,15,246638,17.0,0.0
2,16,1233052,5.0,1.0
3,20,986505,4.0,5.0
4,21,246635,1.0,0.0
5,22,246629,0.0,1.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,38.0,7.0
1,15,17.0,0.0
2,16,5.0,1.0
3,20,4.0,5.0
4,21,1.0,0.0
5,22,0.0,1.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,38.0
1,15,Released,17.0
2,16,Released,5.0
3,20,Released,4.0
4,21,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,20,21,22
Status,,,,,,
Executing,7.0,0.0,1.0,5.0,0.0,1.0
Released,38.0,17.0,5.0,4.0,1.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,20,21,22
0,Executing,7.0,0.0,1.0,5.0,0.0,1.0
1,Released,38.0,17.0,5.0,4.0,1.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,20,21,22
0,Executing,7.0,0.0,1.0,5.0,0.0,1.0
1,Released,38.0,17.0,5.0,4.0,1.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()